# Momentum Decile Performance

The [Fama-French](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_10_port_form_pr_12_2_daily.html) data-set has returns for portfolios constructed out of each decile of prior returns. Labeled LO_PRIOR, PRIOR_2..9 and HI_PRIOR, they represent portfolios who's prior returns were the lowest through to the highest.

Here, we plot the returns of these portfolios to get a sense for how they have behaved through time.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
#startDt <- (famaFrench$MomentumDaily() %>% summarize(MAX = min(TIME_STAMP)) %>% collect())$MAX[[1]]
startDt <- as.Date("2019-01-01")

valueWtd <- famaFrench$MomentumDaily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    mutate(group=1) %>% 
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

print(head(valueWtd))
print(tail(valueWtd))

           HI_PRIOR LO_PRIOR PRIOR_2 PRIOR_3 PRIOR_4 PRIOR_5 PRIOR_6 PRIOR_7
2019-01-02   0.0010   0.0337  0.0163  0.0187  0.0108  0.0070  0.0067  0.0001
2019-01-03  -0.0260  -0.0145 -0.0027 -0.0233 -0.0183 -0.0260 -0.0185 -0.0333
2019-01-04   0.0452   0.0446  0.0474  0.0433  0.0370  0.0328  0.0348  0.0299
2019-01-07   0.0235   0.0376  0.0201  0.0130  0.0066  0.0083  0.0063  0.0041
2019-01-08   0.0138   0.0017  0.0107  0.0136  0.0092  0.0039  0.0079  0.0092
2019-01-09   0.0048   0.0148  0.0142  0.0109  0.0115  0.0033  0.0038  0.0032
           PRIOR_8 PRIOR_9
2019-01-02 -0.0062 -0.0078
2019-01-03 -0.0221 -0.0262
2019-01-04  0.0295  0.0347
2019-01-07  0.0032  0.0071
2019-01-08  0.0154  0.0100
2019-01-09  0.0012  0.0059
           HI_PRIOR LO_PRIOR PRIOR_2 PRIOR_3 PRIOR_4 PRIOR_5 PRIOR_6 PRIOR_7
2019-06-21  -0.0074  -0.0006 -0.0035 -0.0022 -0.0018  0.0035  0.0001 -0.0039
2019-06-24   0.0005  -0.0251 -0.0067 -0.0105 -0.0110 -0.0054 -0.0057  0.0001
2019-06-25  -0.0156  -0.0006 -0.0056 -0.0

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)